In [1]:
import pandas as pd
import time
import datetime
import scipy.stats as st
import numpy as np
import os
import ipyparallel as ipp

In [2]:
# prepare data
#msgat = pd.read_pickle(r'../records/sample/msgat_dataframe')
total_msg = pd.read_pickle(r'../records/sample/sample_msg_tfidf_pickle')
# drop useless columns
total_msg.drop(columns=['type', 'status', 'isSend', 'splitwords','msg'], inplace=True)
# test a subsample of total_msg
total_msg.sort_values('createTime',inplace= True)
total_msg['intTime'] = total_msg.createTime.diff().dt.total_seconds().fillna(0).cumsum()
total_msg.sort_values(['talker','createTime'],inplace= True,ascending=True)
total_msg.index = pd.RangeIndex(len(total_msg.index))
total_msg['talker'] = total_msg['talker'].astype('category')
total_msg['sender'] = total_msg['sender'].astype('category')
total_msg['displayname'] = total_msg['displayname'].astype('category')
total_msg['nickname'] = total_msg['nickname'].astype('category')
total_msg['createDate'] = total_msg['createTime'].dt.date
total_msg['createDate'] = total_msg['createDate'].astype('category')
total_msg = total_msg.iloc[:1000]

In [3]:
twindow = 3  # 这个应该用在全局的totalmsg表上，每次处理相应窗口的数据
divideN = 1500  # 对timediff进行缩放

w_temp = 0.1
temporal_scale = 4.686

In [4]:
def sumDicts(dict1,dict2):
    """Sum dict1 with dict2, only 2.
    """
    return {k: dict1.get(k, 0) + dict2.get(k, 0) for k in dict1.keys() | dict2.keys()}

def sumTfidf(tfidfList):
    """Sum the new computed tfidf
    Args:
        tfidfList: A list of tfidf
    Return:
        the return list: expanded tfidf dict
    """
    rList = {}
    for tfidf in tfidfList:
        rList = sumDicts(tfidf, rList)
    return rList

def probMultiTfidf(probArray, tfidfList):
    """Multiply the context tfidf with probability in the same thread.
    Args:
        probArray(Array): probability in the same thread of msg context
        tfidfList(List): the tfidfList of msg context
    Returns:
        The return tuple
    """
    return ([{k: v for k, v in zip(tfidfList[i].keys(), probArray[i] * np.array(list(tfidfList[i].values())))} for i in
             range(len(probArray))])
def tempProbC(row,t_scale, contextdf,w_temp):
    """Expand the context-free info of msgi with temporal context.

    Args:
        row(Series): One raw of msgdf.itertuples() to be expanded
        t_scale(float): The scale of normal distribution trained before
        contextdf(dataframe): The dataframe of msgs between [DATE - 1DAY,DATE + 1DAY]

    Returns:
        The return vector. The expand vector repretation of msgi with info from its temporal context.

    """
    msgrowid = row[0]
    talker = row[3]
    msgdate = row[8]#intTime

    tempContext = contextdf[contextdf.index != msgrowid]#这里使用index
    timediff = tempContext['intTime'].values - msgdate
    timediff = timediff / divideN
    probArray = st.norm.pdf(timediff, scale=t_scale)
    tfidfList = tempContext['tfidf'].tolist()
    newTfidf = probMultiTfidf(probArray, tfidfList)
    result = {k:(v * w_temp) for k, v in sumTfidf(newTfidf).items()}
    filename = talker[:-9]
    #print(filename)
    #print(result)
    with open(r'../records/sample/tempExpand/temp_expand{}'.format(filename),'a') as f:
        f.write("{0},{1}\n".format(msgrowid,result))
def mExpandTempP(lt):
    msgdate,talker = lt
    #maxdate = datetime.datetime(msgdate.year, msgdate.month, msgdate.day) + datetime.timedelta(hours=24 + twindow)
    #mindate = datetime.datetime(msgdate.year, msgdate.month, msgdate.day) - datetime.timedelta(hours=twindow)
    #maxdate = (maxdate - minGlobalDate).total_seconds()
    #mindate = (mindate - minGlobalDate).total_seconds()
    #还需要限制在同一个聊天室中
    #contextdf = total_msg[(total_msg.intTime > mindate) & (total_msg.intTime < maxdate) & (total_msg.talker == talker)]
    targetdf = total_msg[(total_msg.createDate == msgdate) & (total_msg.talker == talker)]
    contextdf = targetdf
    #[tempProb(row,temporal_scale,contextdf,w_temp) for row in targetdf.itertuples(index=False)]        
    #targetdf.apply(lambda row: tempProbC(row,temporal_scale,contextdf,w_temp),axis=1)
    #follow speeds up
    for row in targetdf.itertuples():
        tempProbC(row,temporal_scale,contextdf,w_temp)

# 并行代码

In [5]:
c = ipp.Client()
c.ids

[0, 1]

In [6]:
dview = c[:]

In [7]:
lview = c.load_balanced_view()
lview.block = True

push变量

In [8]:
dview.push(dict(total_msg = total_msg,twindow = twindow,divideN = divideN,w_temp = w_temp,temporal_scale = temporal_scale))

<AsyncResult: _push>

Engines放入需要的包

In [9]:
%%px 
import pandas as pd
import time
import datetime
import scipy.stats as st
import numpy as np
import os
import sys

In [10]:
%%px
def sumDicts(dict1,dict2):
    """Sum dict1 with dict2, only 2.
    """
    return {k: dict1.get(k, 0) + dict2.get(k, 0) for k in dict1.keys() | dict2.keys()}

def sumTfidf(tfidfList):
    """Sum the new computed tfidf
    Args:
        tfidfList: A list of tfidf
    Return:
        the return list: expanded tfidf dict
    """
    rList = {}
    for tfidf in tfidfList:
        rList = sumDicts(tfidf, rList)
    return rList

def probMultiTfidf(probArray, tfidfList):
    """Multiply the context tfidf with probability in the same thread.
    Args:
        probArray(Array): probability in the same thread of msg context
        tfidfList(List): the tfidfList of msg context
    Returns:
        The return tuple
    """
    return ([{k: v for k, v in zip(tfidfList[i].keys(), probArray[i] * np.array(list(tfidfList[i].values())))} for i in
             range(len(probArray))])
def tempProbC(row,t_scale, contextdf,w_temp):
    """Expand the context-free info of msgi with temporal context.

    Args:
        row(Series): One raw of msgdf.itertuples() to be expanded
        t_scale(float): The scale of normal distribution trained before
        contextdf(dataframe): The dataframe of msgs between [DATE - 1DAY,DATE + 1DAY]

    Returns:
        The return vector. The expand vector repretation of msgi with info from its temporal context.

    """
    msgrowid = row[0]
    talker = row[3]
    msgdate = row[8]#intTime

    tempContext = contextdf[contextdf.index != msgrowid]#这里使用index
    timediff = tempContext['intTime'].values - msgdate
    timediff = timediff / divideN
    probArray = st.norm.pdf(timediff, scale=t_scale)
    tfidfList = tempContext['tfidf'].tolist()
    newTfidf = probMultiTfidf(probArray, tfidfList)
    result = {k:(v * w_temp) for k, v in sumTfidf(newTfidf).items()}
    filename = talker[:-9]
    #print(filename)
    #print(result)
    #with open(r'../records/sample/test/temp_expand{}'.format(filename),'a') as f:
    f = open(r'../records/sample/test/temp_expand{}'.format(filename),'a')#z这个地方需要显示指明要写入的文件的位置
    f.write("{0},{1}\n".format(msgrowid,result))

In [11]:
dview.pull('tempProbC').get()#已经放入了函数

got unknown result: f091b91c-aab49562140cc382bf30c708


[<function tempProbC(row, t_scale, contextdf, w_temp)>,
 <function tempProbC(row, t_scale, contextdf, w_temp)>]

# 执行

现在就发现其中的问题了，创建的engines的位置在'C:\\Users\\Eric'而不是当前的文件所在的位置

In [18]:
dview.apply(lambda :os.getcwd()).get()

['C:\\Users\\Eric', 'C:\\Users\\Eric']

In [17]:
dview.apply(lambda :'i test').get()

['i test', 'i test']

In [ ]:
parallel_result = lview.map(mExpandTempP, [(msgdate,talker) for msgdate in total_msg['createDate'].unique() for talker in total_msg['talker'].unique()])